In [2]:
import tensorflow as tf
import os
import time
import numpy as np
tf.compat.v1.disable_eager_execution()
print(tf.executing_eagerly())

False


In [51]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
MAX_PKT_BYTES = 50 * 50
MAX_PKT_NUM = 100
_batch_size = 4096

_pkt_num = 20
_pkt_bytes = 256
def _parse_sparse_example(example_proto):
    features = {
        'sparse': tf.io.SparseFeature(index_key=['idx1', 'idx2'],
                                        value_key='val',
                                        dtype=tf.int64,
                                        size=[MAX_PKT_NUM, MAX_PKT_BYTES]),
        'label': tf.io.FixedLenFeature([], dtype=tf.int64),
        'byte_len': tf.io.FixedLenFeature([], dtype=tf.int64),
        'last_time': tf.io.FixedLenFeature([], dtype=tf.float32),
    }
    batch_sample = tf.io.parse_example(example_proto, features)
    sparse_features = batch_sample['sparse']
    labels = batch_sample['label']
    sparse_features = tf.sparse.slice(sparse_features, start=[0, 0], size=[_pkt_num, _pkt_bytes])
    dense_features = tf.sparse.to_dense(sparse_features)
    dense_features = tf.cast(dense_features, tf.float32) / 255. # type: ignore
    return dense_features, labels


def _generate_ds(path_dir, use_cache=True):
    assert os.path.isdir(path_dir)
    ds = tf.data.Dataset.list_files(os.path.join(path_dir, '*.tfrecord'), shuffle=True)
    ds = ds.interleave(
        lambda x: tf.data.TFRecordDataset(x).map(_parse_sparse_example),
        cycle_length=AUTOTUNE, 
        block_length=8,
        num_parallel_calls=AUTOTUNE
    ) # type: ignore
    # ds = ds.batch(_batch_size, drop_remainder=False)
    if use_cache:
        ds = ds.cache()
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_path='D:\\ids2018\\tfrecord\\train'
valid_path='D:\\ids2018\\tfrecord\\valid'
test_path='D:\\ids2018\\tfrecord\\test'


In [56]:
valid_ds = _generate_ds(valid_path)
iterator = tf.compat.v1.data.make_one_shot_iterator(valid_ds)
next_element = iterator.get_next()
# with tf.compat.v1.Session() as sess:
#     try:
#         value = sess.run(next_element)
#         print(value[1])
#         # 对 value 进行操作
#     except tf.errors.OutOfRangeError:
#         pass  # 数据集中的所有数据已经被遍历

In [59]:
count = 0
while True:
    with tf.compat.v1.Session() as sess:
        try:
            value = sess.run(next_element)
            # count+=value[1].shape[0]
            # print(count)
            # 对 value 进行操作
            if count%256 == 0:
                print(count)
            count+=1
        except Exception:
            break  # 数据集中的所有数据已经被遍历
print(count)

0
256
512
768
1024
1280
1536
1792
2048
2304


KeyboardInterrupt: 

In [3]:
valid_ds = _generate_ds(valid_path)
# 转换为ndarray

features = []
lables = []
for feature, label in valid_ds:
    features.append(feature.numpy())
    lables.append(label.numpy())

features= np.array(features)
lables = np.array(lables)

Instructions for updating:
No similar op available at this time.


In [8]:
np.save('D:\\ids2018\\tfrecord\\valid\\features.npy', features)
np.save('D:\\ids2018\\tfrecord\\valid\\labels.npy', lables)

In [3]:
test_ds = _generate_ds(test_path)

i = 0
features_path = 'D:\\ids2018\\tfrecord\\test\\features.npy'
labels_path = 'D:\\ids2018\\tfrecord\\test\\labels.npy'
features_mm = np.memmap(features_path, mode='w+', dtype='float32', shape=(732689, 20, 256, 1))
labels_mm = np.memmap(labels_path, mode='w+', dtype='int64', shape=(732689,))

for feature, label in test_ds:
    feature = feature.numpy().reshape(-1, 20, 256, 1)
    label = label.numpy()
    if feature.shape[0] == 4096:
        features_mm[i*4096:(i+1)*4096] = feature
        labels_mm[i*4096:(i+1)*4096] = label
    else:
        features_mm[i*4096:732689] = feature
        labels_mm[i*4096:732689] = label
    i+=1
    print(i)



Instructions for updating:
No similar op available at this time.
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179


In [4]:
del features_mm
del labels_mm

In [12]:
mm = np.memmap(features_path, dtype='float32', mode='r', shape=(732689, 20, 256, 1))
print(mm.shape)

(732689, 20, 256, 1)
